In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from fibsem import utils, acquire, movement, calibration
from fibsem.structures import ImageSettings, BeamType
from pprint import pprint
import matplotlib.pyplot as plt
from fibsem import milling
import numpy as np


In [28]:
protocol_path = r"C:\Users\Admin\Github\fibsem\example\protocol\protocol.yaml"
microscope, settings = utils.setup_session(protocol_path=protocol_path, session_path= r"D:\cyril\tin-balls-sweep")

2022-10-24 11:58:49,041 — root — INFO — connect_to_microscope:31 — Microscope client connecting to [10.0.0.1]
2022-10-24 11:58:49,046 — root — INFO — connect_to_microscope:34 — Microscope client connected to [10.0.0.1]
2022-10-24 11:58:49,047 — root — INFO — setup_session:264 — Finished setup for session: slice-and-view-cyril_2022-10-24.11-58-49AM


Client connecting to [10.0.0.1:7520]...
Client connected to [10.0.0.1:7520]


In [29]:
settings.image.save_path

# settings.image.save = True
# eb_image = acquire.new_image(microscope, settings.image)
# eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)


'D:\\cyril\\tin-balls-sweep'

In [30]:
# settings
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.beams.electron_beam.optical_mode.value = "Immersion"
microscope.detector.mode.value = "BackscatterElectrons"
settings.system.stage.tilt_flat_to_electron = 0

import numpy as np
microscope.beams.electron_beam.angular_correction.angle.value = np.deg2rad(-38)

# # LUT: inverse


In [31]:
from autoscript_sdb_microscope_client.enumerations import DetectorMode, DetectorType
# microscope.detector.set_type_mode(type=DetectorType.ETD, mode = DetectorMode.BACKSCATTER_ELECTRONS)

In [32]:
from fibsem.structures import MicroscopeSettings
from autoscript_sdb_microscope_client import SdbMicroscopeClient


def run_data_collection(microscope: SdbMicroscopeClient, settings: MicroscopeSettings, dwell_times: list[float], step: int):
    
    settings.image.save = True
    settings.image.autocontrast = False

    for i, dwell_time in enumerate(dwell_times):

        settings.image.dwell_time = dwell_time
        print(f"{i} - dwell_time: {dwell_time}")

        # take multiple short dwell time images
        if dwell_time < 2e-6:
            for j in range(7):
                settings.image.label = f"EB_{step}_{dwell_time}_{j}"
                print(i, j, dwell_time, settings.image.label)
                acquire.new_image(microscope, settings.image)
        else:
            settings.image.label = f"EB_{step}_{dwell_time}_0"
            print(i, 0, dwell_time, settings.image.label)
            # take a single high dwell time
            acquire.new_image(microscope, settings.image)
        
        print("-"*50)

In [33]:
# settings.image.save_path = r"D:\cyril\slice-and-view-cyril_2022-10-14.02-18-00PM"
print(settings.image.save_path)

D:\cyril\tin-balls-sweep


In [19]:

# start wiith cleaning cross section to clean the area
centre_x = 0
centre_y = 0
width = 40e-6
height = 2.5e-6
depth = 10e-6

microscope.imaging.set_active_view(BeamType.ION.value)
microscope.imaging.set_active_device(BeamType.ION.value)
microscope.patterning.clear_patterns()

pattern = microscope.patterning.create_cleaning_cross_section(center_x=centre_x, center_y=centre_y, width=width, height=height, depth=depth)
pattern.scan_direction = "BottomToTop"

In [35]:
# settings
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.detector.contrast.limits

Limits(min=0, max=1)

In [36]:
state = calibration.get_current_microscope_state(microscope)
# detector contrast brightness
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
contrast = microscope.detector.contrast.value
brightness = microscope.detector.brightness.value

In [37]:
# microscope.beams.electron_beam.beam_current.value = 0.1e-9
print(brightness, contrast)

0.41373929001664533 0.8310507240125296


In [38]:
acquire.autocontrast(microscope, BeamType.ELECTRON)

2022-10-24 11:58:54,977 — root — INFO — autocontrast:27 — automatically adjusting contrast...


In [39]:
start_x, end_x = -8e-6, 33e-6
start_y, end_y = -4e-6, -4e-6
depth = 8e-6
offset = 5e-9


N_SLICES = 500
START_IDX = 22
# for i in range(START_IDX, N_SLICES):

microscope.imaging.set_active_view(BeamType.ION.value)
microscope.imaging.set_active_device(BeamType.ION.value)
microscope.patterning.clear_patterns()
pattern = microscope.patterning.create_line(start_x=start_x, start_y=start_y, end_x=end_x, end_y=end_y, depth=depth)
print(pattern)

In [40]:

settings.image.hfw = 30e-6
settings.image.resolution = "6144x4096"
dwell_times = [0.5e-6, 1e-6, 3e-6, 5e-6, 8e-6, 20e-6]
settings.image.save = True
settings.image.autocontrast = False
settings.image.gamma.enabled = False

start_x, end_x = -8e-6, 33e-6
start_y, end_y = -4e-6, -4e-6
depth = 8e-6
offset = 5e-9

N_SLICES = 500
START_IDX = 0
for i in range(START_IDX, N_SLICES):

    print(f" ---------------- SLICE {i}/{N_SLICES} ---------------- ")
    
    settings.image.dwell_time = 0.2e-6
    settings.image.save = False
    # slice
    if i > START_IDX:
        microscope.imaging.set_active_view(BeamType.ION.value)
        microscope.imaging.set_active_device(BeamType.ION.value)
        microscope.patterning.clear_patterns()
        pattern = microscope.patterning.create_line(start_x=start_x, start_y=start_y, end_x=end_x, end_y=end_y, depth=depth)
        print(pattern)

        # run
        milling.run_milling(microscope, milling_current=5.6e-9)
    
    # view
    calibration.auto_discharge_beam(microscope, settings.image, n_iterations=5)

    settings.image.autocontrast = False
    run_data_collection(microscope, settings, dwell_times, i)

    start_y += offset
    end_y  += offset
    
    # manually adjust working distance
    wd_diff = offset * np.sin(np.deg2rad(38))
    microscope.beams.electron_beam.working_distance.value -= wd_diff #4e-3# 3.995e-3 

    if i % 50 == 0 and i != START_IDX:
        acquire.autocontrast(microscope, BeamType.ELECTRON)


2022-10-24 11:58:59,032 — root — INFO — auto_discharge_beam:142 — Bring me Thanos!
2022-10-24 11:58:59,075 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


 ---------------- SLICE 0/500 ---------------- 


2022-10-24 11:58:59,563 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-24 11:59:00,023 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-24 11:59:00,457 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-24 11:59:00,906 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-24 11:59:01,355 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.
2022-10-24 11:59:07,894 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


0 - dwell_time: 5e-07
0 0 5e-07 EB_0_5e-07_0


2022-10-24 11:59:21,734 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


0 1 5e-07 EB_0_5e-07_1


In [ ]:
# microscope.auto_functions.run_auto_focus()
from autoscript_sdb_microscope_client.structures import RunAutoFocusSettings

focus_settings = RunAutoFocusSettings()

microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.auto_functions.run_auto_focus()
microscope.specimen.stage.link()


# microscope.auto_functions.run_auto_stigmator()

In [ ]:
settings.image.hfw = 30e-6
settings.image.resolution = "6144x4096"
dwell_times = [0.3e-6, 0.5e-6, 1e-6, 3e-6, 5e-6, 8e-6]
settings.image.save = True
settings.image.autocontrast = True
settings.image.gamma.enabled = False
settings.image.dwell_time = 0.5e-6
settings.image.label = "4um"
microscope.beams.electron_beam.working_distance.value = 4e-3# 3.995e-3 
acquire.new_image(microscope, settings.image)

In [ ]:
# # dynamic focus : 38

# 4e-3 - (1000*5e-9)

In [ ]:
settings.image.save_path

In [ ]:
# multichem / GIS query

## Tin Ball Data Collection


In [1]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, movement, calibration
from fibsem.structures import ImageSettings, BeamType
from pprint import pprint
import matplotlib.pyplot as plt
from fibsem import milling
import numpy as np
from copy import deepcopy
import pandas as pd
import os


protocol_path = r"C:\Users\Admin\Github\fibsem\example\protocol\protocol.yaml"
microscope, settings = utils.setup_session(protocol_path=protocol_path, session_path= r"D:\cyril\tin-balls-sweep")

2022-10-24 17:10:24,251 — root — INFO — connect_to_microscope:31 — Microscope client connecting to [10.0.0.1]
2022-10-24 17:10:24,256 — root — INFO — connect_to_microscope:34 — Microscope client connected to [10.0.0.1]
2022-10-24 17:10:24,256 — root — INFO — setup_session:264 — Finished setup for session: slice-and-view-cyril_2022-10-24.05-10-24PM


Client connecting to [10.0.0.1:7520]...
Client connected to [10.0.0.1:7520]


In [ ]:
# settings
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.beams.electron_beam.optical_mode.value = "Immersion"
microscope.detector.mode.value = "BackscatterElectrons"
settings.system.stage.tilt_flat_to_electron = 0


# microscope.beams.electron_beam.angular_correction.angle.value = np.deg2rad(-38)

# 1kv voltage

# # LUT: inverse


In [2]:
settings.image.save_path

'D:\\cyril\\tin-balls-sweep'

In [4]:
# pixelsize (nm), # pixelsize = hfw/n_pixels_x
# hfws
n_pixels_x = 3072 
pixel_size = np.array([1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16, 18, 20])
hfws = pixel_size * n_pixels_x * 1e-9 # nm

print(hfws)

[3.0720e-06 6.1440e-06 9.2160e-06 1.2288e-05 1.5360e-05 1.8432e-05
 2.4576e-05 3.0720e-05 3.6864e-05 4.3008e-05 4.9152e-05 5.5296e-05
 6.1440e-05]


In [6]:
# beam current, resolution, dwell time
voltages = [2e3]
# beam_currents = [50e-12, 0.1e-9,  0.2e-9, 0.4e-9, 0.8e-9] 
beam_currents = [3.2e-9]
dwell_times = [2e-6]
resolution = "3072x2048"

settings.image.resolution = resolution
settings.image.autocontrast = False
settings.image.gamma.enabled = False
settings.image.save = True

N_IMAGES = 1

counter = 0
all_params = []
for m, voltage in enumerate(voltages):
    microscope.beams.electron_beam.high_voltage.value = voltage
    for i, beam_current in enumerate(beam_currents):

        # change beam current
        microscope.beams.electron_beam.beam_current.value = beam_current
        # microscope.auto_functions.run_auto_stigmator()
        # microscope.auto_functions.run_auto_focus()
        # calibration.auto_discharge_beam(microscope, settings.image)
        input("confirm beam focus / stigmation")

        for j, hfw in enumerate(hfws):

            acquire.autocontrast(microscope, BeamType.ELECTRON)

            for k, dwell_time in enumerate(dwell_times):
                
                for n in range(N_IMAGES):

                    # set image settings
                    label = f"{counter:02d}.sweep_{m}.{i}.{j}.{k}.{n}.{voltage:.2e}.{beam_current:.2e}.{hfw:.2e}.{dwell_time:.2e}"
                    settings.image.hfw = hfw
                    settings.image.dwell_time = dwell_time
                    settings.image.label = label

                    pprint(settings.image.label)

                    params = {"voltage": voltage, 
                        "beam_current": beam_current,
                        "hfw": hfw,
                        "dwell_time": dwell_time,
                        "n_image": n,
                        "label": label,
                        "count": counter,
                        }

                    all_params.append(deepcopy(params))

                    image = acquire.new_image(microscope, settings.image)
                    counter+=1 
    
# save data
df = pd.DataFrame.from_dict(all_params)
df.to_csv(os.path.join(settings.image.save_path, "data.csv"))
display(df)

# image_count.sweep.m.i.j.k.n.voltage.beam_current.hfw.dwell_time


2022-10-24 17:39:15,515 — root — INFO — autocontrast:27 — automatically adjusting contrast...
2022-10-24 17:39:16,149 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


'00.sweep_0.0.0.0.0.2.00e+03.3.20e-09.3.07e-06.2.00e-06'


2022-10-24 17:39:29,603 — root — INFO — autocontrast:27 — automatically adjusting contrast...
2022-10-24 17:39:30,346 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


'01.sweep_0.0.1.0.0.2.00e+03.3.20e-09.6.14e-06.2.00e-06'


2022-10-24 17:39:43,777 — root — INFO — autocontrast:27 — automatically adjusting contrast...
2022-10-24 17:39:44,540 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


...

2022-10-24 17:41:52,773 — root — INFO — autocontrast:27 — automatically adjusting contrast...
2022-10-24 17:41:54,012 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


'11.sweep_0.0.11.0.0.2.00e+03.3.20e-09.5.53e-05.2.00e-06'


2022-10-24 17:42:07,458 — root — INFO — autocontrast:27 — automatically adjusting contrast...
2022-10-24 17:42:08,459 — root — INFO — acquire_image:186 — acquiring new ELECTRON image.


'12.sweep_0.0.12.0.0.2.00e+03.3.20e-09.6.14e-05.2.00e-06'


,voltage,beam_current,hfw,dwell_time,n_image,label,count
0,2000.0,3.200000e-09,0.000003,0.000002,0,00.sweep_0.0.0.0.0.2.00e+03.3.20e-09.3.07e-06....,0
1,2000.0,3.200000e-09,0.000006,0.000002,0,01.sweep_0.0.1.0.0.2.00e+03.3.20e-09.6.14e-06....,1
2,2000.0,3.200000e-09,0.000009,0.000002,0,02.sweep_0.0.2.0.0.2.00e+03.3.20e-09.9.22e-06....,2
3,2000.0,3.200000e-09,0.000012,0.000002,0,03.sweep_0.0.3.0.0.2.00e+03.3.20e-09.1.23e-05....,3
4,2000.0,3.200000e-09,0.000015,0.000002,0,04.sweep_0.0.4.0.0.2.00e+03.3.20e-09.1.54e-05....,4
5,2000.0,3.200000e-09,0.000018,0.000002,0,05.sweep_0.0.5.0.0.2.00e+03.3.20e-09.1.84e-05....,5
6,2000.0,3.200000e-09,0.000025,0.000002,0,06.sweep_0.0.6.0.0.2.00e+03.3.20e-09.2.46e-05....,6
7,2000.0,3.200000e-09,0.000031,0.000002,0,07.sweep_0.0.7.0.0.2.00e+03.3.20e-09.3.07e-05....,7
8,2000.0,3.200000e-09,0.000037,0.000002,0,08.sweep_0.0.8.0.0.2.00e+03.3.20e-09.3.69e-05....,8
9,2000.0,3.200000e-09,0.000043,0.000002,0,09.sweep_0.0.9.0.0.2.00e+03.3.20e-09.4.30e-05....,9


'D:\\cyril\\tin-balls-sweep'